Project Title¶

Data Engineering Capstone Project
Project Summary

--describe your project at a high level--

The project follows the follow steps:

Step 1: Scope the Project and Gather Data
Step 2: Explore and Assess the Data
Step 3: Define the Data Model
Step 4: Run ETL to Model the Data
Step 5: Complete Project Write Up

In [13]:
import re
import pandas as pd
import psycopg2
from collections import defaultdict
from datetime import datetime, timedelta
from pyspark.sql.functions import udf
from pyspark.sql import SparkSession

### 1.scope the project and gather the data

1. Datasets contain million of lines and scope to provide migrant people where they area(city,state,country) going to (destination) and population at destination area.
2. Male and Female ratio present in areas and average temperature present there.
3. Their Mode of travel and based on which visa type.

#### Resources

1. Immigration data of USA
2. Us-cities demographic  data from opensoft
3. Global Temparature dataset from koggle


Mainly to prepare Analytic table


### Step 2: Explore and Assess the Data

In [14]:
# Getting Temperature data 
temperature_data = '../../data2/GlobalLandTemperaturesByCity.csv'
df = pd.read_csv(temperature_data, sep=',')

In [15]:
# print number of rows in dataset
print(df.count())

dt                               8599212
AverageTemperature               8235082
AverageTemperatureUncertainty    8235082
City                             8599212
Country                          8599212
Latitude                         8599212
Longitude                        8599212
dtype: int64


In [16]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1743-11-01,6.068,1.737,Århus,Denmark,57.05N,10.33E
1,1743-12-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
2,1744-01-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
3,1744-02-01,NaN,NaN,Århus,Denmark,57.05N,10.33E
4,1744-03-01,NaN,NaN,Århus,Denmark,57.05N,10.33E


In [17]:
# Read Immigration data which is csv format,just a sample for now then we work on big data set using spark
path= 'immigration_data_sample.csv'
df_immigration_pd = pd.read_csv(path, sep=',')

In [18]:
df_immigration_pd.head()

,Unnamed: 0,cicid,i94yr,i94mon,i94cit,i94res,i94port,arrdate,i94mode,i94addr,...,entdepu,matflag,biryear,dtaddto,gender,insnum,airline,admnum,fltno,visatype
0,2027561,4084316.0,2016.0,4.0,209.0,209.0,HHW,20566.0,1.0,HI,...,NaN,M,1955.0,07202016,F,NaN,JL,5.658267e+10,00782,WT
1,2171295,4422636.0,2016.0,4.0,582.0,582.0,MCA,20567.0,1.0,TX,...,NaN,M,1990.0,10222016,M,NaN,*GA,9.436200e+10,XBLNG,B2
2,589494,1195600.0,2016.0,4.0,148.0,112.0,OGG,20551.0,1.0,FL,...,NaN,M,1940.0,07052016,M,NaN,LH,5.578047e+10,00464,WT
3,2631158,5291768.0,2016.0,4.0,297.0,297.0,LOS,20572.0,1.0,CA,...,NaN,M,1991.0,10272016,M,NaN,QR,9.478970e+10,00739,B2
4,3032257,985523.0,2016.0,4.0,111.0,111.0,CHM,20550.0,3.0,NY,...,NaN,M,1997.0,07042016,F,NaN,NaN,4.232257e+10,LAND,WT


In [19]:
# Read demographic data 
path = 'us-cities-demographics.csv'
df_demographic_pd = pd.read_csv(path,sep=";")
df_demographic_pd.head()

,City,State,Median Age,Male Population,Female Population,Total Population,Number of Veterans,Foreign-born,Average Household Size,State Code,Race,Count
0,Silver Spring,Maryland,33.8,40601.0,41862.0,82463,1562.0,30908.0,2.60,MD,Hispanic or Latino,25924
1,Quincy,Massachusetts,41.0,44129.0,49500.0,93629,4147.0,32935.0,2.39,MA,White,58723
2,Hoover,Alabama,38.5,38040.0,46799.0,84839,4819.0,8229.0,2.58,AL,Asian,4759
3,Rancho Cucamonga,California,34.5,88127.0,87105.0,175232,5821.0,33878.0,3.18,CA,Black or African-American,24437
4,Newark,New Jersey,34.6,138040.0,143873.0,281913,5829.0,86253.0,2.73,NJ,White,76402


#### Accessing Data with spark for better performance


In [20]:
# creating the spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.\
config("spark.jars.packages","saurfang:spark-sas7bdat:2.0.0-s_2.11")\
.enableHiveSupport().getOrCreate()

### Step 2.Explore and Assess the data
#### explore


##### Immigration Data 
This data comes from the US National Tourism and Trade Office. We drop all rows having null or invalid values for i94port which is know for destination area.

In [21]:
path_immigration_data= '../../data/18-83510-I94-Data-2016/i94_apr16_sub.sas7bdat'
spark_immigration_data = spark.read.format('com.github.saurfang.sas.spark').load(path_immigration_data)
spark_immigration_data.show()

+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|cicid| i94yr|i94mon|i94cit|i94res|i94port|arrdate|i94mode|i94addr|depdate|i94bir|i94visa|count|dtadfile|visapost|occup|entdepa|entdepd|entdepu|matflag|biryear| dtaddto|gender|insnum|airline|        admnum|fltno|visatype|
+-----+------+------+------+------+-------+-------+-------+-------+-------+------+-------+-----+--------+--------+-----+-------+-------+-------+-------+-------+--------+------+------+-------+--------------+-----+--------+
|  6.0|2016.0|   4.0| 692.0| 692.0|    XXX|20573.0|   null|   null|   null|  37.0|    2.0|  1.0|    null|    null| null|      T|   null|      U|   null| 1979.0|10282016|  null|  null|   null| 1.897628485E9| null|      B2|
|  7.0|2016.0|   4.0| 254.0| 276.0|    ATL|20551.0|    1.0|     AL|   null|  25.0|    3.0|  1.0|20130811|     SE

In [22]:
# Creating dictionary of i94port which are of valid format
re_compile_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
valid_i94port = {}
with open('i94port_valid.txt') as f:
     for line in f:
            match = re_compile_obj.search(line)
            valid_i94port[match[1]]=[match[2]]



In [23]:
# clean i94 immigration data using valid ports data obtained above
spark_immigration_valid_port = spark_immigration_data.filter(spark_immigration_data.i94port.isin(list(valid_i94port)))

#### Temperature Data

In [24]:
path = "../../data2/GlobalLandTemperaturesByCity.csv"
spark_temp = spark.read.format("csv").option("header",True).option("delimiter",",").load(path)
spark_temp.show()

+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01| 14.050999999999998|        

In [25]:
#Adding a column "i94port" to spark_temp dataset

@udf
def get_column(city):
    for key in valid_i94port:
        if key.lower() in valid_i94port[key][0].lower():
            return key
        



In [26]:
spark_temp.show()

+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01| 14.050999999999998|        

In [33]:
spark_temp.count()

3490

In [30]:
# Number of records after removing null values in AverageTemperature column
spark_temp = spark_temp.filter(spark_temp.AverageTemperature != 'null')
spark_temp.count()

8235082

In [31]:
spark_temp.show()

+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01| 14.050999999999998|                        1.347|Århus|Denmark|  57.05N|   10.33E|
|1744-07-01|             16.082|                        1.396|Århus|Denmark|  57.05N|   10.33E|
|1744-09-01| 12.780999999999999|                        1.454|Århus|Denmark|  57.05N|   10.33E|
|1744-10-01|               7.95|                         1.63|Århus|Denmark|  57.05N|   10.33E|
|1744-11-01|  4.638999999999999|        

In [32]:
spark_temp = spark_temp.dropDuplicates(['City','Country'])
spark_temp.show()

+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        City|           Country|Latitude|Longitude|
+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|
|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|
|1825-01-01|  26.069000000000003|                         2.16|     Bintulu|          Malaysia|   2.41N|  113.30E|
|1825-01-01|              26.517|           2.5839999999999996| Butterworth|          Malaysia|   5.63N|  100.09E|
|1845-01-01|              24.995|                        1.871|      Cainta|       Philippines|  15.27N|  120.83E|
|1825-01-01|              24.753|           2.1519999999999997|      Ciamis|    

In [34]:
spark_temp.count()

3490

#### Us-Cities-demographics


In [35]:
path = 'us-cities-demographics.csv'
spark_us_cities = spark.read.format("csv").option("header", True).option("delimiter",";").load(path)
spark_us_cities.show()

+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|            City|         State|Median Age|Male Population|Female Population|Total Population|Number of Veterans|Foreign-born|Average Household Size|State Code|                Race| Count|
+----------------+--------------+----------+---------------+-----------------+----------------+------------------+------------+----------------------+----------+--------------------+------+
|   Silver Spring|      Maryland|      33.8|          40601|            41862|           82463|              1562|       30908|                   2.6|        MD|  Hispanic or Latino| 25924|
|          Quincy| Massachusetts|      41.0|          44129|            49500|           93629|              4147|       32935|                  2.39|        MA|               White| 58723|
|          Hoover|       Alabama|      38.5|      

In [36]:
spark_us_cities = spark_us_cities.dropDuplicates(["City","State"])
spark_us_cities.count()

596

### Step 3.Define the Data Model

3.1 Conceptual Model

First dimentional model contains data from i94 immigration dataset
- i94yr = 4 digit year
- i94mon = numeric month
- i94cit = 3 digit code of origin city
- i94port = 3 character code of destination city
- arrdate = arrival date
- i94mode = 1 digit travel code
- depdate = departure date
- i94visa = reason for immigration

Second dimentional model contain data from GobalTemperature dataset

- City = city name
- Country = Country name
- latitude = latitude 
- longitude = longitude
- avaerage temperature = Average temperature pesent in specific latitude and longitude
- i94port = 3 character code of destination city (added using immigration data)


Third dimentional model contains data from us-cities-demographics dataset

- City = city name
- State = state name
- Male population - male population in that city
- Female population - Female Population in that city
- race = people of specific race
- Average household size = avaerage house hold size

****Fist Fact Table1**** 

##### Contain information by joining immigration data to temperature data on i94port and again joing to us-cities-demog   raphics data on city name####

- i94yr = 4 digit year code
- i94mon =  digit month code(1,,,,12)
- i94cit = 3 digit of origin city
- i94port = 3 character code of destination city
- arrdata = arrival data
- i94mode = mode of travel
- gender = gender
- airline = airline code
- deptdate = departure date
- i94visa = type of reason for visa
- Average Temperature= average temperature in city
- Male Population = male population in that city
- Female population = femle population
- race = people of specific race



****Fist Fact Table2**** 

##### contain information by joining selected columns from fact table1 and us-cities-demographics dataset on city column.Where city were only from USA

- city = city name
- male population = male population
- female population = female population
- average household size = average household size
- reason = reason
- i94port = 3 character destional city
- i94month = numeric month


****Necessary steps to pipline the data (datamodel specified above)****

- 1.Clean the immigration dataset by removing duplicates and null values in i94port port and making spark dataframe done this in above stemps

- 2.Clean temperature dataset on addng i94port column with valid state code and removing duplicate state and country names.

- 3.Create first dimentional table after cleaning and removing duplicates and null values in i94immigration data

- 4.Create second dimentional table on global temperature dataset with column "i94port".

- 5.Create third dimentional table on us-cities-demographics dataset with column names specified above and removing duplicate cities and statues

- 6.Create fact table by joining immigration and temperature dimension tables on i94port and again join demographics dimentional table on city and write to parquet file partitioned by i94port and city.

**Run ETL to Model the Data**

***Step1***

In [37]:
# Path to I94 immigration data 
# Here only april data is loaded but we can load whole millions of data using 
# path = '../../data/18-83510-I94-Data-2016/*.sas7bdat'
path = '../../data/18-83510-I94-Data-2016/i94_jun16_sub.sas7bdat'

In [38]:
# we have already loaded april data and done some calculation , now using june data 
spark_immigration_data = spark.read.format('com.github.saurfang.sas.spark').load(path)

***Cleaning Data***

In [39]:
# Creating dictionary of i94port which are of valid format for june month
re_compile_obj = re.compile(r'\'(.*)\'.*\'(.*)\'')
valid_i94port = {}
with open('i94port_valid.txt') as f:
     for line in f:
            match = re_compile_obj.search(line)
            valid_i94port[match[1]]=[match[2]]


In [40]:
spark_immigration_data = spark_immigration_data.filter(spark_immigration_data.i94port.isin(list(valid_i94port)))

In [41]:
# selecting i94port column which contain only valid port
spark_immigration_data.select(["i94port"]).show()

+-------+
|i94port|
+-------+
|    SFR|
|    SFR|
|    HOU|
|    HOU|
|    BOS|
|    NEW|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    NYC|
|    TAM|
|    BOS|
|    BOS|
+-------+
only showing top 20 rows



In [42]:
# Create immigration dimentional table
immigration_dim = spark_immigration_data.select(["i94yr", "i94mon", "i94cit", "i94port", "arrdate", "i94mode", "depdate", "i94visa"])
immigration_dim.show()

+------+------+------+-------+-------+-------+-------+-------+
| i94yr|i94mon|i94cit|i94port|arrdate|i94mode|depdate|i94visa|
+------+------+------+-------+-------+-------+-------+-------+
|2016.0|   6.0| 254.0|    SFR|20623.0|    1.0|   null|    3.0|
|2016.0|   6.0| 254.0|    SFR|20623.0|    1.0|   null|    3.0|
|2016.0|   6.0| 127.0|    HOU|20617.0|    1.0|20679.0|    2.0|
|2016.0|   6.0| 127.0|    HOU|20617.0|    1.0|20679.0|    2.0|
|2016.0|   6.0| 101.0|    BOS|20606.0|    1.0|20608.0|    1.0|
|2016.0|   6.0| 101.0|    NEW|20606.0|    1.0|20639.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20610.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20618.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20610.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20610.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20610.0|    1.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20610.0|    2.0|
|2016.0|   6.0| 101.0|    NYC|20606.0|    1.0|20626.0| 

In [43]:
# writing data to separet folder in parquet formate and partition by i94port and i94month
immigration_dim.write.mode("append").partitionBy("i94port").parquet("/results/immigration.parquet")

***step2***

In [44]:
# path to temperature data
fname = '../../data2/GlobalLandTemperaturesByCity.csv'
temperature_dim = spark.read.format("csv").option("header",True).option("delimiter", ",").load(fname)
temperature_dim.show()

+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|        dt| AverageTemperature|AverageTemperatureUncertainty| City|Country|Latitude|Longitude|
+----------+-------------------+-----------------------------+-----+-------+--------+---------+
|1743-11-01|              6.068|           1.7369999999999999|Århus|Denmark|  57.05N|   10.33E|
|1743-12-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-01-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-02-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-03-01|               null|                         null|Århus|Denmark|  57.05N|   10.33E|
|1744-04-01| 5.7879999999999985|           3.6239999999999997|Århus|Denmark|  57.05N|   10.33E|
|1744-05-01|             10.644|           1.2830000000000001|Århus|Denmark|  57.05N|   10.33E|
|1744-06-01| 14.050999999999998|        

In [45]:
temperature_dim = temperature_dim.dropDuplicates(["City","Country"])
temperature_dim.show()

+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        City|           Country|Latitude|Longitude|
+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|
|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|
|1825-01-01|  26.069000000000003|                         2.16|     Bintulu|          Malaysia|   2.41N|  113.30E|
|1825-01-01|              26.517|           2.5839999999999996| Butterworth|          Malaysia|   5.63N|  100.09E|
|1845-01-01|              24.995|                        1.871|      Cainta|       Philippines|  15.27N|  120.83E|
|1825-01-01|              24.753|           2.1519999999999997|      Ciamis|    

In [46]:
temperature_dim.count()

3490

***Cleaning Step***

In [47]:
# Adding column i94port to temperature_dim table
temperature_dim = temperature_dim.withColumn("i94port", get_column(temperature_dim.City))
temperature_dim.show()

+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        City|           Country|Latitude|Longitude|i94port|
+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|    ALC|
|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|    ALC|
|1825-01-01|  26.069000000000003|                         2.16|     Bintulu|          Malaysia|   2.41N|  113.30E|    ALC|
|1825-01-01|              26.517|           2.5839999999999996| Butterworth|          Malaysia|   5.63N|  100.09E|    ALC|
|1845-01-01|              24.995|                        1.871|      Cainta|       Philippines|  15.27N|  120.83E|    ALC|
|1825-01-01|    

In [48]:
# Removing all null values in Average Temperature column
temperature_dim = temperature_dim.filter(temperature_dim["AverageTemperature"] != 'null')
temperature_dim.show()

+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        City|           Country|Latitude|Longitude|i94port|
+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|    ALC|
|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|    ALC|
|1825-01-01|  26.069000000000003|                         2.16|     Bintulu|          Malaysia|   2.41N|  113.30E|    ALC|
|1825-01-01|              26.517|           2.5839999999999996| Butterworth|          Malaysia|   5.63N|  100.09E|    ALC|
|1845-01-01|              24.995|                        1.871|      Cainta|       Philippines|  15.27N|  120.83E|    ALC|
|1825-01-01|    

In [49]:
# Removing all duplicates in city and country columns 
temperature_dim = temperature_dim.dropDuplicates(["City","Country"])


In [50]:
# Extracting columns Required columns from temperature_dim table to create dimentional table
temperature_table = temperature_dim.select(["City", "Country", "Latitude", "Longitude","AverageTemperature","i94port"])
temperature_table = temperature_dim.withColumnRenamed("City","city")
temperature_table.show()

+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|        dt|  AverageTemperature|AverageTemperatureUncertainty|        city|           Country|Latitude|Longitude|i94port|
+----------+--------------------+-----------------------------+------------+------------------+--------+---------+-------+
|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|    ALC|
|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|    ALC|
|1825-01-01|  26.069000000000003|                         2.16|     Bintulu|          Malaysia|   2.41N|  113.30E|    ALC|
|1825-01-01|              26.517|           2.5839999999999996| Butterworth|          Malaysia|   5.63N|  100.09E|    ALC|
|1845-01-01|              24.995|                        1.871|      Cainta|       Philippines|  15.27N|  120.83E|    ALC|
|1825-01-01|    

In [51]:
temperature_table.count()

3490

In [52]:
# writing table to parquet formate
temperature_table.write.mode("append").partitionBy("i94port").parquet("/results/temperature.parquet")

***Step3***

In [53]:
# path to us-cities-demographic datase
path = "us-cities-demographics.csv"
demographics_table = spark.read.format("csv").option("header",True).option("delimiter",";").load(path)


In [54]:
# Extracting required columns to model the dimention table
from pyspark.sql.functions import col
demographics_dim = demographics_table.select(col("City").alias("city"), col("State").alias("state"), col("Male Population").alias("male_population"), col("Female Population").alias("female_population"), col("Average Household Size").alias("average_household_size"))
demographics_dim.show()

+----------------+--------------+---------------+-----------------+----------------------+
|            city|         state|male_population|female_population|average_household_size|
+----------------+--------------+---------------+-----------------+----------------------+
|   Silver Spring|      Maryland|          40601|            41862|                   2.6|
|          Quincy| Massachusetts|          44129|            49500|                  2.39|
|          Hoover|       Alabama|          38040|            46799|                  2.58|
|Rancho Cucamonga|    California|          88127|            87105|                  3.18|
|          Newark|    New Jersey|         138040|           143873|                  2.73|
|          Peoria|      Illinois|          56229|            62432|                   2.4|
|        Avondale|       Arizona|          38712|            41971|                  3.18|
|     West Covina|    California|          51629|            56860|                  3.56|

In [55]:
# writing data to results directory in parquet format
demographics_dim.write.mode("append").partitionBy("city").parquet("/results/demographics.parquet")

***step3***

In [56]:
# Creating Fact table from Dimenstion tables which were created above

# Create views for each table 
demographics_dim.createOrReplaceTempView("demographics_view")
temperature_table.createOrReplaceTempView("temperature_view")
immigration_dim.createOrReplaceTempView("immigration_view")


In [57]:
# Create immigration_temperature_table by joining immigration dimention table and temperature_table dimention table
immigration_temperature_table = immigration_dim.join(temperature_table, on=["i94port"])

In [58]:
immigration_temperature_table.show()

+-------+------+------+------+-------+-------+-------+-------+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|i94port| i94yr|i94mon|i94cit|arrdate|i94mode|depdate|i94visa|        dt|  AverageTemperature|AverageTemperatureUncertainty|        city|           Country|Latitude|Longitude|
+-------+------+------+------+-------+-------+-------+-------+----------+--------------------+-----------------------------+------------+------------------+--------+---------+
|    ALC|2016.0|   6.0| 103.0|20613.0|    3.0|20626.0|    2.0|1743-11-01|               3.264|                        1.665|   Allentown|     United States|  40.99N|   74.56W|
|    ALC|2016.0|   6.0| 103.0|20613.0|    3.0|20626.0|    2.0|1779-11-01|0.011999999999999985|                        2.714|      Atyrau|        Kazakhstan|  47.42N|   50.92E|
|    ALC|2016.0|   6.0| 103.0|20613.0|    3.0|20626.0|    2.0|1825-01-01|  26.069000000000003|                         2

In [59]:
immigration_temperature_table.printSchema()

root
 |-- i94port: string (nullable = true)
 |-- i94yr: double (nullable = true)
 |-- i94mon: double (nullable = true)
 |-- i94cit: double (nullable = true)
 |-- arrdate: double (nullable = true)
 |-- i94mode: double (nullable = true)
 |-- depdate: double (nullable = true)
 |-- i94visa: double (nullable = true)
 |-- dt: string (nullable = true)
 |-- AverageTemperature: string (nullable = true)
 |-- AverageTemperatureUncertainty: string (nullable = true)
 |-- city: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Latitude: string (nullable = true)
 |-- Longitude: string (nullable = true)



In [60]:
# selecting required column for fact table changing column name to readable format
fact_table = immigration_temperature_table.select(col("i94yr").alias("year"),
                                                col("i94mon").alias("month"),
                                                col("i94cit").alias("city_code"), 
                                                col("city").alias("city"),
                                                col("Country").alias("country"),
                                                col("i94port").alias("i94port"),
                                                col("arrdate").alias("arrival_date"),
                                                col("i94mon").alias("tavel_mode"),
                                                col("depdate").alias("departure_date"),
                                                col("i94visa").alias("reason"),
                                                col("AverageTemperature").alias("averagetemperature"),
                                                col("Latitude").alias("latitude"),
                                                col("Longitude").alias("longitude")
                                               )
fact_table.printSchema()

root
 |-- year: double (nullable = true)
 |-- month: double (nullable = true)
 |-- city_code: double (nullable = true)
 |-- city: string (nullable = true)
 |-- country: string (nullable = true)
 |-- i94port: string (nullable = true)
 |-- arrival_date: double (nullable = true)
 |-- tavel_mode: double (nullable = true)
 |-- departure_date: double (nullable = true)
 |-- reason: double (nullable = true)
 |-- averagetemperature: string (nullable = true)
 |-- latitude: string (nullable = true)
 |-- longitude: string (nullable = true)



In [61]:
fact_table.show()

+------+-----+---------+------------+------------------+-------+------------+----------+--------------+------+--------------------+--------+---------+
|  year|month|city_code|        city|           country|i94port|arrival_date|tavel_mode|departure_date|reason|  averagetemperature|latitude|longitude|
+------+-----+---------+------------+------------------+-------+------------+----------+--------------+------+--------------------+--------+---------+
|2016.0|  6.0|    103.0|   Allentown|     United States|    ALC|     20613.0|       6.0|       20626.0|   2.0|               3.264|  40.99N|   74.56W|
|2016.0|  6.0|    103.0|      Atyrau|        Kazakhstan|    ALC|     20613.0|       6.0|       20626.0|   2.0|0.011999999999999985|  47.42N|   50.92E|
|2016.0|  6.0|    103.0|     Bintulu|          Malaysia|    ALC|     20613.0|       6.0|       20626.0|   2.0|  26.069000000000003|   2.41N|  113.30E|
|2016.0|  6.0|    103.0| Butterworth|          Malaysia|    ALC|     20613.0|       6.0|      

In [62]:
fact_table.count()

2966500

In [63]:
fact_table.write.mode("append").partitionBy(["i94port","city"]).parquet("/results/fact_table.parquet")
print("success")

success


### Create another fact table to analyse immigration people in USA

- Create separate  tables for fact_table and demographics tables 
- Then make sure in fact_table we have only USA cities 
- Join two tables and make sure we dont have any repeated or null values after joining

In [64]:
# select required columns from fact_table created above
fact_table_immigration = fact_table.select(["city", "i94port", "month", "reason"])

In [65]:
# Get all city names in USA and make sure wont have duplicate cities in the list
usa_cities_list = [row["city"] for row in demographics_dim.collect()]
set_usa_list = set(usa_cities_list)
len(set_usa_list)

567

In [66]:
# function to verify whether city in usa_cities list or not
@udf
def get_city(city):
    '''
    input: city name
    
    output: if city in set_usa_list return city else return null
    '''
    if city in set_usa_list:
        return city


In [67]:
# Add usa_city column to fact_table_immigration table
fact_table_immigration = fact_table_immigration.withColumn("usa_city", get_city(fact_table_immigration.city))

In [68]:
# drop column usa_city 
fact_table_immigration = fact_table_immigration.drop('usa_city')

In [69]:
# Drop duplicate rows in city column
fact_table_immigration = fact_table_immigration.dropDuplicates(["city"])

In [70]:
# results obtainer
fact_table_immigration.show()

+------------+-------+-----+------+
|        city|i94port|month|reason|
+------------+-------+-----+------+
|     Antwerp|    ALC|  6.0|   2.0|
|    Araruama|    ALC|  6.0|   2.0|
|   Bangalore|    ALC|  6.0|   2.0|
|       Benxi|    ALC|  6.0|   2.0|
|   Cajamarca|    ALC|  6.0|   2.0|
|  Cicalengka|    ALC|  6.0|   2.0|
|     Copiapo|    ALC|  6.0|   2.0|
|     Da Nang|    ALC|  6.0|   2.0|
|Dos Hermanas|    ALC|  6.0|   2.0|
|    Fengshan|    ALC|  6.0|   2.0|
|    Girardot|    ALC|  6.0|   2.0|
|     Hanover|    ALC|  6.0|   2.0|
|    Horlivka|    ALC|  6.0|   2.0|
|    Jaboatão|    ALC|  6.0|   2.0|
|   Kisangani|    ALC|  6.0|   2.0|
|   Magdeburg|    ALC|  6.0|   2.0|
|       Mainz|    ALC|  6.0|   2.0|
|    Montería|    ALC|  6.0|   2.0|
|     Morelia|    ALC|  6.0|   2.0|
|    Namangan|    ALC|  6.0|   2.0|
+------------+-------+-----+------+
only showing top 20 rows



In [71]:
# Join fact_table_immigration and demographics_dim table on city 
fact_table_immigration =  fact_table_immigration.join(demographics_dim, on=["city"])


In [72]:
# Drop duplicate values present in table and remove duplicate values.
fact_table_immigration = fact_table_immigration.dropDuplicates(['city'])
fact_table_immigration.show()

+---------------+-------+-----+------+--------------+---------------+-----------------+----------------------+
|           city|i94port|month|reason|         state|male_population|female_population|average_household_size|
+---------------+-------+-----+------+--------------+---------------+-----------------+----------------------+
|      Worcester|    ALC|  6.0|   2.0| Massachusetts|          90951|            93855|                  2.43|
|     Charleston|    ALC|  6.0|   2.0|South Carolina|          63956|            71568|                   2.4|
|         Corona|    ALC|  6.0|   2.0|    California|          79749|            84493|                  2.97|
|    Springfield|    ALC|  6.0|   2.0| Massachusetts|          74744|            79592|                  2.81|
|          Tempe|    ALC|  6.0|   2.0|       Arizona|          91350|            84476|                  2.44|
|North Las Vegas|    ALC|  6.0|   2.0|        Nevada|         116350|           118443|                  3.31|
|

In [73]:
fact_table_immigration.write.mode("append").partitionBy(["i94port","city"]).parquet("/results/fact_table_immigration.parquet")
print("success")

success


### 4.2 Data Quality Checks###

Explain the data quality checks you'll perform to ensure the pipeline ran as expected. These could include:

- Integrity constraints on the relational database (e.g., unique key, data type, etc.)
- Unit tests for the scripts to ensure they are doing the right thing
- Source/Count checks to ensure completeness

#### Run Quality checks

In [74]:
def quality_check(df, description):
    '''
    Input: Spark dataframe, description of Spark datafram
    Output: Print outcome of data quality check
    '''
    
    result = df.count()
    if result == 0:
        print("Data quality check failed for {} with zero records".format(description))
    else:
        print("Data quality check passed for {} with {} records".format(description, result))
    return 0

In [75]:
# Perform data quality check
quality_check(immigration_dim, "immigration table")
quality_check(temperature_table, "temperature table")
quality_check(demographics_dim, "demographics table")
quality_check(fact_table, "fact table")
quality_check(fact_table_immigration, "fact_table_immigration table")


Data quality check passed for immigration table with 3508813 records
Data quality check passed for temperature table with 3490 records
Data quality check passed for demographics table with 2891 records
Data quality check passed for fact table with 2966500 records
Data quality check passed for fact_table_immigration table with 252 records


0

### 4.3 Data dictionary 
Create a data dictionary for your data model. For each field, provide a brief description of what the data is and where it came from. You can include the data dictionary in the notebook or in a separate file.

### fact_table 
this will contain data by joining i94 immigration to gobaltemperature dataset

##### columns

- i94yr     AS year 
- i94mon    AS month
- i94cit    AS city_code
- city      AS city
- Country   AS country
- i94port   AS i94port
- arrrdate  AS arrival_date
- i94visa   AS reason
- i94mod    AS travel_mode
- depdate   AS departure_date
- Average Temperature AS averagetemperature
- Latitude  AS latitude
- Longitude AS longitude 

#### fact_table_immigration

##### Columns

- city AS city
- i94port AS i94port
- i94mon AS month
- i94visa AS reason
- state AS state
- Male Population AS male_population
- Female Population AS female_population
- Average HouseHold Size AS average_household_size

#### 1.Dimentional Table
Contain Data from i94immigration data

Columns

- i94yr = 4 digit year
- i94mon = numeric month
- i94cit = 3 digit code of origin city
- i94port = 3 character code of destination city
- arrdate = arrival date
- i94mode = 1 digit travel code
- depdate = departure date
- i94visa = reason for immigration

### 2.Dimentional Table
Contain data from global temperature data

Columns

- 94port = 3 character code of destination city (mapped from cleaned up immigration data)
- AverageTemperature = average temperature
- City = city name
- Country = country name
- Latitude= latitude
- Longitude = longitude

### 3.Dimentional Table
Contain data from us-cities-demograhics dataset

Columns
- 

- city  AS city
- state AS state
- Male Population AS male_population
- Female Population AS female_population
- Average HouseHold Size AS average_household_size

## Step 5: Complete Project Write Up
1 Clearly state the rationale for the choice of tools and technologies for the project.

- we used Spark for this project since it can easily handle multiple file formats (json,SAS, csv,text, etc) that contain large i.e 1M amounts of data. Its mainly because ***Schema-on-Read***(no database storage required) which is main advantage over datawarehouse.Spark is know for **Lazy Evaluation** means whenever you want data,then it run quiries and take data from disk to memory. Spark SQL was used to process the input files into dataframes and manipulated via standard SQL join operations to create the tables.

2 Propose how often the data should be updated and why.

- Data is updated for every months. we can analyse data by pullin for every months.

3 Write a description of how you would approach the problem differently under the following scenarios:
 * The data was increased by 100x.
 
 - AWS provides AmazonRedshift using which we can analyze millions of data workloads easily
 
4 The data populates a dashboard that must be updated on a daily basis by 7am every day.
 
 - Using Airflow DAG we can trigger the task for every 7am. if task fails sending emails can be done.
 
 - DataQualitycheck pulgins can be included in our task for every load data we analyse.
 
5 The database needed to be accessed by 100+ people.
 
 - Using Redshift we can solve this problem, it has autoscaling capabilities and good read performance
 